In [16]:
import io
import time
import requests
import datetime
import pandas as pd

from util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)

In [17]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Functions.get_previous_date(date_tday)
date_bf_yday = Functions.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 11, 6),
 datetime.date(2020, 11, 5),
 datetime.date(2020, 11, 4))

In [18]:
# Today url for regions file
url_tday = Functions.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Functions.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Functions.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201106.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201105.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201104.csv')

In [19]:
wait = True
if wait:
    while True:
        response = requests.get(url_tday)
        if response.status_code == 200:
            print('Presenti dati odierni')
            date_last = date_tday
            csv = response.content
            df_raw_r_1 = pd.read_csv(io.StringIO(csv.decode('utf-8')))
            df_raw_r_0 = pd.read_csv(url_yday)
            break
        else:
            print('Status code: {}. Retrying...'.format(status_code))
        time.sleep(20)
else:
    print('Dati di ieri')
    date_last = date_yday
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)

Presenti dati odierni


In [20]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-06T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3871,268,4139,43777,...,3942,4878,36993,4549,39162,50296,89458,1117280,703606,NaN
1,2020-11-06T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,154,13,167,2001,...,16,129,1600,195,3481,482,3963,44359,27094,NaN
2,2020-11-06T17:00:00,ITA,3,Lombardia,45.466794,9.190347,5563,570,6133,111233,...,4842,9934,107463,18118,190116,52831,242947,3172359,1960088,NaN
3,2020-11-06T17:00:00,ITA,5,Veneto,45.434905,12.338452,1184,169,1353,42584,...,3218,3297,25612,2543,25565,46527,72092,2417660,937280,NaN
4,2020-11-06T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,260,44,304,6451,...,376,542,6162,435,11675,1677,13352,567637,238032,NaN
5,2020-11-06T17:00:00,ITA,7,Liguria,44.411493,8.932699,1315,78,1393,10299,...,469,1127,20881,1879,24320,10132,34452,475685,244325,NaN
6,2020-11-06T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1673,177,1850,31880,...,1754,1953,28559,4752,46964,20077,67041,1695309,897938,In seguito a verifica sui dati comunicati nei ...
7,2020-11-06T17:00:00,ITA,9,Toscana,43.769231,11.255889,1303,209,1512,37928,...,1958,2592,16738,1502,46541,11139,57680,1180350,780994,effettuati 19323 tamponi rapidi antigenici di ...
8,2020-11-06T17:00:00,ITA,10,Umbria,43.106758,12.388247,326,53,379,8506,...,496,767,4532,174,3841,9750,13591,320893,185144,NaN
9,2020-11-06T17:00:00,ITA,11,Marche,43.616760,13.518875,444,60,504,8736,...,613,697,7379,1037,17400,256,17656,338947,199235,NaN


In [21]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-05T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3698,249,3947,40027,...,2397,3171,36086,4520,37804,46776,84580,1095992,687826,NaN
1,2020-11-05T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,160,12,172,1980,...,28,114,1493,189,3375,459,3834,43640,26872,NaN
2,2020-11-05T17:00:00,ITA,3,Lombardia,45.466794,9.190347,5318,522,5840,106684,...,7791,8822,102502,17987,183660,49353,233013,3125958,1935518,NaN
3,2020-11-05T17:00:00,ITA,5,Veneto,45.434905,12.338452,1162,164,1326,39393,...,3183,3264,25560,2516,25539,43256,68795,2397655,930576,NaN
4,2020-11-05T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,268,44,312,6067,...,398,546,6008,423,11257,1553,12810,561085,236298,NaN
5,2020-11-05T17:00:00,ITA,7,Liguria,44.411493,8.932699,1236,76,1312,9911,...,756,1208,20239,1863,23424,9901,33325,469913,241779,NaN
6,2020-11-05T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1588,177,1765,30211,...,2002,2180,28403,4712,45945,19146,65091,1674462,890032,In seguito a verifica sui dati sono stati elim...
7,2020-11-05T17:00:00,ITA,9,Toscana,43.769231,11.255889,1387,202,1589,35893,...,2018,2273,16136,1470,44373,10715,55088,1164607,771596,effettuati 17941 tamponi rapidi antigenici di ...
8,2020-11-05T17:00:00,ITA,10,Umbria,43.106758,12.388247,319,50,369,8020,...,553,768,4270,165,3674,9150,12824,316038,182633,NaN
9,2020-11-05T17:00:00,ITA,11,Marche,43.616760,13.518875,421,56,477,8150,...,629,698,7298,1034,16759,200,16959,335105,197050,NaN


In [22]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-11-06T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3871,268,4139,43777,...,3942,4878,36993,4549,39162,50296,89458,1117280,703606,NaN
1,2020-11-06T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,154,13,167,2001,...,16,129,1600,195,3481,482,3963,44359,27094,NaN
2,2020-11-06T17:00:00,ITA,3,Lombardia,45.466794,9.190347,5563,570,6133,111233,...,4842,9934,107463,18118,190116,52831,242947,3172359,1960088,NaN
3,2020-11-06T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,582,52,634,8845,...,-405,974,11757,789,17436,4589,22025,554506,244625,NaN
4,2020-11-06T17:00:00,ITA,5,Veneto,45.434905,12.338452,1184,169,1353,42584,...,3218,3297,25612,2543,25565,46527,72092,2417660,937280,NaN
5,2020-11-06T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,260,44,304,6451,...,376,542,6162,435,11675,1677,13352,567637,238032,NaN
6,2020-11-06T17:00:00,ITA,7,Liguria,44.411493,8.932699,1315,78,1393,10299,...,469,1127,20881,1879,24320,10132,34452,475685,244325,NaN
7,2020-11-06T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1673,177,1850,31880,...,1754,1953,28559,4752,46964,20077,67041,1695309,897938,In seguito a verifica sui dati comunicati nei ...
8,2020-11-06T17:00:00,ITA,9,Toscana,43.769231,11.255889,1303,209,1512,37928,...,1958,2592,16738,1502,46541,11139,57680,1180350,780994,effettuati 19323 tamponi rapidi antigenici di ...
9,2020-11-06T17:00:00,ITA,10,Umbria,43.106758,12.388247,326,53,379,8506,...,496,767,4532,174,3841,9750,13591,320893,185144,NaN


In [23]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-11-05T17:00:00,ITA,1,Piemonte,45.073274,7.680687,3698,249,3947,...,2397,3171,36086,4520,37804,46776,84580,1095992,687826,NaN
1,17,2020-11-05T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,160,12,172,...,28,114,1493,189,3375,459,3834,43640,26872,NaN
2,8,2020-11-05T17:00:00,ITA,3,Lombardia,45.466794,9.190347,5318,522,5840,...,7791,8822,102502,17987,183660,49353,233013,3125958,1935518,NaN
3,19,2020-11-05T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,559,48,607,...,770,1082,10387,780,16565,4486,21051,548082,242716,NaN
4,18,2020-11-05T17:00:00,ITA,5,Veneto,45.434905,12.338452,1162,164,1326,...,3183,3264,25560,2516,25539,43256,68795,2397655,930576,NaN
5,5,2020-11-05T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,268,44,312,...,398,546,6008,423,11257,1553,12810,561085,236298,NaN
6,7,2020-11-05T17:00:00,ITA,7,Liguria,44.411493,8.932699,1236,76,1312,...,756,1208,20239,1863,23424,9901,33325,469913,241779,NaN
7,4,2020-11-05T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,1588,177,1765,...,2002,2180,28403,4712,45945,19146,65091,1674462,890032,In seguito a verifica sui dati sono stati elim...
8,15,2020-11-05T17:00:00,ITA,9,Toscana,43.769231,11.255889,1387,202,1589,...,2018,2273,16136,1470,44373,10715,55088,1164607,771596,effettuati 17941 tamponi rapidi antigenici di ...
9,16,2020-11-05T17:00:00,ITA,10,Umbria,43.106758,12.388247,319,50,369,...,553,768,4270,165,3674,9150,12824,316038,182633,NaN


In [24]:
# Print raw columns
# print(df_r_1.columns, df_r_0.columns)
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rf_1 = df_r_1.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0 = df_r_0.drop(cols_to_drop, axis=1).sort_values(by='codice_regione').reset_index(drop=True)
# df_rf_1.columns, df_rf_0.columns

In [25]:
df_rf_0 = df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)
df_rf_0;

In [26]:
df_r = df_rf_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-06T17:00:00,1,Piemonte,3871,268,4139,43777,47916,3942,4878,36993,4549,39162,50296,89458,1117280,703606,4341375
1,2020-11-06T17:00:00,2,Valle d'Aosta,154,13,167,2001,2168,16,129,1600,195,3481,482,3963,44359,27094,125501
2,2020-11-06T17:00:00,3,Lombardia,5563,570,6133,111233,117366,4842,9934,107463,18118,190116,52831,242947,3172359,1960088,10103969
3,2020-11-06T17:00:00,4,Trentino-Alto Adige,582,52,634,8845,9479,-405,974,11757,789,17436,4589,22025,554506,244625,1074819
4,2020-11-06T17:00:00,5,Veneto,1184,169,1353,42584,43937,3218,3297,25612,2543,25565,46527,72092,2417660,937280,4907704
5,2020-11-06T17:00:00,6,Friuli Venezia Giulia,260,44,304,6451,6755,376,542,6162,435,11675,1677,13352,567637,238032,1211357
6,2020-11-06T17:00:00,7,Liguria,1315,78,1393,10299,11692,469,1127,20881,1879,24320,10132,34452,475685,244325,1543127
7,2020-11-06T17:00:00,8,Emilia-Romagna,1673,177,1850,31880,33730,1754,1953,28559,4752,46964,20077,67041,1695309,897938,4467118
8,2020-11-06T17:00:00,9,Toscana,1303,209,1512,37928,39440,1958,2592,16738,1502,46541,11139,57680,1180350,780994,3722729
9,2020-11-06T17:00:00,10,Umbria,326,53,379,8506,8885,496,767,4532,174,3841,9750,13591,320893,185144,880285


In [27]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-06T17:00:00,1,Piemonte,3871,268,4139,43777,47916,3942,4878,36993,4549,39162,50296,89458,1117280,703606,4341375,367
1,2020-11-06T17:00:00,2,Valle d'Aosta,154,13,167,2001,2168,16,129,1600,195,3481,482,3963,44359,27094,125501,20
2,2020-11-06T17:00:00,3,Lombardia,5563,570,6133,111233,117366,4842,9934,107463,18118,190116,52831,242947,3172359,1960088,10103969,983
3,2020-11-06T17:00:00,4,Trentino-Alto Adige,582,52,634,8845,9479,-405,974,11757,789,17436,4589,22025,554506,244625,1074819,106
4,2020-11-06T17:00:00,5,Veneto,1184,169,1353,42584,43937,3218,3297,25612,2543,25565,46527,72092,2417660,937280,4907704,825
5,2020-11-06T17:00:00,6,Friuli Venezia Giulia,260,44,304,6451,6755,376,542,6162,435,11675,1677,13352,567637,238032,1211357,175
6,2020-11-06T17:00:00,7,Liguria,1315,78,1393,10299,11692,469,1127,20881,1879,24320,10132,34452,475685,244325,1543127,209
7,2020-11-06T17:00:00,8,Emilia-Romagna,1673,177,1850,31880,33730,1754,1953,28559,4752,46964,20077,67041,1695309,897938,4467118,516
8,2020-11-06T17:00:00,9,Toscana,1303,209,1512,37928,39440,1958,2592,16738,1502,46541,11139,57680,1180350,780994,3722729,415
9,2020-11-06T17:00:00,10,Umbria,326,53,379,8506,8885,496,767,4532,174,3841,9750,13591,320893,185144,880285,70


In [28]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [29]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-06T17:00:00,3,Lombardia,5563,570,6133,111233,117366,4842,9934,...,57.99,4549,1112,46401,24570,131,4961,21.41,0.09832,2.40
1,2020-11-06T17:00:00,1,Piemonte,3871,268,4139,43777,47916,3942,4878,...,73.02,3750,1707,21288,15780,29,907,22.91,0.11236,2.06
2,2020-11-06T17:00:00,15,Campania,1677,180,1857,60339,62196,4148,4508,...,42.15,4073,620,23897,18412,40,320,18.86,0.07791,1.35
3,2020-11-06T17:00:00,5,Veneto,1184,169,1353,42584,43937,3218,3297,...,20.48,3191,33,20005,6704,27,52,16.48,0.06718,1.47
4,2020-11-06T17:00:00,12,Lazio,2511,234,2745,43939,46684,2394,2699,...,31.33,2379,-36,28744,21319,26,279,9.39,0.04601,1.03
5,2020-11-06T17:00:00,9,Toscana,1303,209,1512,37928,39440,1958,2592,...,50.36,2035,319,15743,9398,32,602,16.46,0.06963,1.55
6,2020-11-06T17:00:00,8,Emilia-Romagna,1673,177,1850,31880,33730,1754,1953,...,34.30,1669,-227,20847,7906,40,156,9.37,0.04372,1.50
7,2020-11-06T17:00:00,19,Sicilia,1157,159,1316,18197,19513,987,1423,...,29.55,975,101,9525,5606,34,402,14.94,0.02864,0.58
8,2020-11-06T17:00:00,7,Liguria,1315,78,1393,10299,11692,469,1127,...,37.32,388,-81,5772,2546,16,642,19.53,0.07303,2.23
9,2020-11-06T17:00:00,4,Trentino-Alto Adige,582,52,634,8845,9479,-405,974,...,49.06,-432,-108,6424,1909,9,1370,15.16,0.09062,2.05


In [30]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)